# Schedule TPC-DS 1 Feature Selection (REP_HIST_SYSMETRIC_SUMMARY)

This notebook is dedicated to dataset profiling. In this notebook, feature selection techniques will be implemented so as to categorize which features belay the most information to address the problem at hand - Workload Prediction. Due to the vast feature space which have been gathered during a workload's execution, manual techniques at determining which are most detrimental is not sufficient. 

Therefore the following work puts emphasis on automated techniques so as to determine out of the vast feature space which are most important to base future models upon. 

In [13]:
# Module Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from operator import itemgetter
from sklearn.metrics import r2_score

### Configuration Cell

Tweak parametric changes from this cell to influence outcome of experiment
* tpcds - Schema upon which to operate test
* debug_mode - Determines whether to plot graphs or not, useful for development purposes
* low_quartile_limit - Lower Quartile threshold to detect outliers
* upper_quartile_limit - Upper Quartile threshold to detect outliers
* test_split - Denotes which Data Split to operate under when it comes to training / validation
* nrows - Number of rows to read from csv file
* top_n_features - Number of top features to focus on
* parallel_degree - Number of parallel threads to train models with

In [14]:
# Experiment Config
tpcds='TPCDS1'
debug_mode=True
low_quartile_limit = 0
upper_quartile_limit = 1
test_split=.3
nrows=None
top_n_features=30
parallel_degree=4

### Read data from file into Pandas Dataframes

In [15]:
# Root path
#root_dir = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds
root_dir = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds

# Open Data
rep_hist_sysmetric_summary_path = root_dir + '/rep_hist_sysmetric_summary.csv'

rep_hist_sysmetric_summary_df = pd.read_csv(rep_hist_sysmetric_summary_path,nrows=nrows)

def prettify_header(headers):
    """
    Cleans header list from unwated character strings
    """
    header_list = []
    [header_list.append(header.replace("(","").replace(")","").replace("'","").replace(",","")) for header in headers]
    return header_list

rep_hist_sysmetric_summary_df.columns = prettify_header(rep_hist_sysmetric_summary_df.columns.values)

MemoryError: 

# Dataset Description

The correlation of resources consumed (y) per snapshot (X) define our feature space. Since the objective here is to attempt to predict what resources will be incurred ahead of time, the problem can be defined as a number of questions:

* Q: What resources can I predict to be in usage at point N in time?
* Q: What resources should I be predicting that accurately portray a schedule's workload?
* Q: What knowledge/data do I have ahead of time which I can use to base my predictions off?

Due to the vast feature space in the available metrics monitored and captured during a workload's execution, it is important to rank which attribute is most beneficial than others. Additionally, it is important to analyze such features individually, and considerate of other features in two types of analysis:

* Univariate Analysis
* Multivariate Analysis

Furthermore, multiple types of feature ranking / analysis techniques ara available, amongst which will be considered:

* Filter Methods
* Wrapper Methods
* Embedded Methods

# Data Preprocessing

We apply a number of preprocessing techniques to the presented dataframes, particularly to normalize and/or scale feature vectors into a more suitable representation for downstream estimators:

Relative Links:
* http://scikit-learn.org/stable/modules/preprocessing.html
* https://machinelearningmastery.com/improve-model-accuracy-with-data-pre-processing/
* https://machinelearningmastery.com/normalize-standardize-time-series-data-python/

### Table Pivots

To better handle the following table, a number of table pivots are made on tables:
* rep_hist_sysmetric_summary

In [ ]:
# Table REP_HIST_SYSMETRIC_SUMMARY
rep_hist_sysmetric_summary_df = rep_hist_sysmetric_summary_df.pivot(index='SNAP_ID', columns='METRIC_NAME', values='AVERAGE')
rep_hist_sysmetric_summary_df.reset_index(inplace=True)
print(rep_hist_sysmetric_summary_df.columns)
rep_hist_sysmetric_summary_df[['SNAP_ID']] = rep_hist_sysmetric_summary_df[['SNAP_ID']].astype(int)
rep_hist_sysmetric_summary_df.sort_values(by=['SNAP_ID'],inplace=True,ascending=True)
print("REP_HIST_SYSMETRIC Shape: " + str(rep_hist_sysmetric_summary_df.shape))

# Refreshing columns with pivoted columns
def convert_list_to_upper(col_list):
    """
    Takes a string and converts elements to upper
    """
    upper_col_list = []
    for col in col_list:
        upper_col_list.append(col.upper())
    return upper_col_list

rep_hist_sysmetric_summary_df.rename(str.upper, inplace=True, axis='columns')

rep_hist_sysmetric_summary_headers = list(rep_hist_sysmetric_summary_df.columns)

# DF Shape
print('Table [REP_HIST_SYSMETRIC_SUMMARY] - ' + str(rep_hist_sysmetric_summary_df.shape))

### Checking for NaN Values

Checking dataframes for potential missing values/data:

In [ ]:
def get_na_columns(df, headers):
    """
    Return columns which consist of NAN values
    """
    na_list = []
    for head in headers:
        if df[head].isnull().values.any():
            na_list.append(head)
    return na_list

print("Table REP_HIST_SYSMETRIC_SUMMARY: " + str(get_na_columns(df=rep_hist_sysmetric_summary_df,headers=rep_hist_sysmetric_summary_headers)) + "\n\n")

def fill_na(df):
    """
    Replaces NA columns with 0s
    """
    return df.fillna(0)

# Populating NaN values with amount '0'
rep_hist_sysmetric_summary_df = fill_na(df=rep_hist_sysmetric_summary_df)

### Checking for Negative Values

A function which retrieves a count per column for nay negative values it might contain

In [ ]:
def count_neg_df(df, headers):
    """
    Return columns with respective negative value count
    """
    neg_list = []
    for head in headers:
        count = 0
        try:
            count = sum(n < 0 for n in df[head].values.flatten())
        except Exception:
            pass
            #print('Non numeric column [' + head + ']')
        if count > 0:
            neg_list.append([head,count])
    return neg_list

def fill_neg(df, headers):
    """
    Sets any data anomilies resulting in negative values to 0
    
    :param headers: list as follows eg: ['column_name', 'negative_count']
    """
    for head in headers:
        try:
            df[df[head[0]] < 0] = 0
        except Exception:
            pass
            #print('Non numeric column [' + head + ']')
    return df

# Check For Negative Values within dataframes
print('---------------WITH NEGATIVE VALUES---------------')
print("Table REP_HIST_SYSMETRIC_SUMMARY: " + str(count_neg_df(df=rep_hist_sysmetric_summary_df,headers=rep_hist_sysmetric_summary_headers)) + "\n\n")

# Replace Negative Values with a minimal threshold of 0
rep_hist_sysmetric_summary_df = fill_neg(df=rep_hist_sysmetric_summary_df,headers=count_neg_df(df=rep_hist_sysmetric_summary_df,headers=rep_hist_sysmetric_summary_headers))

# Check For Negative Values within dataframes
print('\n\n---------------WITHOUT NEGATIVE VALUES---------------')
print("Table REP_HIST_SYSMETRIC_SUMMARY: " + str(count_neg_df(df=rep_hist_sysmetric_summary_df,headers=rep_hist_sysmetric_summary_headers)) + "\n\n")

### Feature Selection

In this step, redundant features are dropped. Features are considered redundant if exhibit a standard devaition of 0 (meaning no change in value).

In [ ]:
def drop_flatline_columns(df):
    columns = df.columns
    flatline_features = []
    for i in range(len(columns)):
        try:
            std = df[columns[i]].std()
            if std == 0:
                flatline_features.append(columns[i])
        except:
            pass
    #
    #print('Features which are considered flatline:\n')
    #for col in flatline_features:
    #    print(col)
    print('\nShape before changes: [' + str(df.shape) + ']')
    df = df.drop(columns=flatline_features)
    print('Shape after changes: [' + str(df.shape) + ']')
    print('Dropped a total [' + str(len(flatline_features)) + ']')
    return df

rep_hist_sysmetric_summary_df = drop_flatline_columns(df=rep_hist_sysmetric_summary_df)

rep_hist_sysmetric_summary_headers = rep_hist_sysmetric_summary_df.columns

## Visualizing Feature Distribution & Skewness

In order to decide between a normalization strategy, it is important to understand the underlying data spread. Understanding of dataset mean, variance, skewness on a per column/feature basis helps determine whether a standardization or normalization strategy should be utilized on the datasets.

### Plotting Data Distribution

To better decide which normalization technique ought to be utilized for the technique at hand, a number of feature columns will be plotted as histograms to better convey the distribution spread.

In [ ]:
def plot_hist(df=None, tpc_type=None, table=None, feature_column=None, bin_size=10):
    """
    Plots histogram distribution
    """
    #
    try:
        df['SNAP_ID'] = df['SNAP_ID'].astype(float)
        df[feature_column] = df[feature_column].astype(float)
        #
        max_val = df[feature_column].max()
        start_snap, end_snap = int(df['SNAP_ID'].min()), int(df['SNAP_ID'].max())
        #
        df[feature_column].hist(bins=10,figsize=(12,8))
        plt.ylabel(feature_column)
        plt.xlabel('Bin Ranges Of ' + str(int(max_val/bin_size)))
        plt.title(tpc_type + ' Table ' + table.upper() + '.' + str(feature_column) + " between " + str(start_snap) + " - " + str(end_snap))
        plt.show()
    except Exception:
        print('Could not plot column: ' + feature_column)

def plot_scatter(df=None, tpc_type=None, table=None, feature_column=None):
    """
    Plots scatter plots vs SNAP_ID
    """
    #
    try:
        df['SNAP_ID'] = df['SNAP_ID'].astype(int)
        df[feature_column] = df[feature_column].astype(int)
        start_snap, end_snap = int(df['SNAP_ID'].min()), int(df['SNAP_ID'].max())
        #
        df.plot.scatter(x='SNAP_ID',
                        y=feature_column,
                        figsize=(12,8))
        plt.ylabel(feature_column)
        plt.xlabel('SNAP ID')
        plt.title(tpc_type + ' Table ' + table.upper() + '.' + str(feature_column) + " between " + str(start_snap) + " - " + str(end_snap))
        plt.show()
    except Exception:
        print('Could not plot column: ' + feature_column)

def plot_boxplot(df=None, tpc_type=None, table=None, feature_columns=None):
    """
    Plots quartile plots to estimate mean and sigma (std dev)
    """
    #
    try:
        for feature_column in feature_columns:
            df[feature_column] = df[feature_column].astype(int)
        df.boxplot(column=feature_columns, figsize=(12,8), grid=True)
        plt.title(tpc_type + ' ' + str(feature_columns))
        plt.show()
    except Exception:
        print('Could not plot column: ' + feature_column)

if debug_mode is False:
    
    # Plotting Histograms of data distribution
    for header in rep_hist_sysmetric_summary_headers:
        print('REP_HIST_SYSMETRIC_SUMMARY - ' + header + ' - OUTLIERS HISTOGRAM')
        plot_hist(df=rep_hist_sysmetric_summary_df, tpc_type=tpcds, table='rep_hist_sysmetric_summary', feature_column=header, bin_size=10)
    
    # Plotting Scatter Plots of data distribution
    for header in rep_hist_sysmetric_summary_headers:
        print('REP_HIST_SYSMETRIC_SUMMARY - ' + header + ' - OUTLIERS SCATTER')
        plot_scatter(df=rep_hist_sysmetric_summary_df, tpc_type=tpcds,table='rep_hist_sysmetric_summary',feature_column=header)

### Outlier Handling

https://machinelearningmastery.com/how-to-identify-outliers-in-your-data/

As can be appreciated from the previous plots, data is heavily skewed on particular (smallest) bins. This skew in the plotted histograms is a result of data point outliers - these need to be evaluated and removed if neccessary.

Following the 3 Standard Deviation Rule, we can categorize our dataset into subsets consisting of the following ranges:
* 0     - 68.27%
* 68.28 - 95.45%
* 95.46 - 99.73%
* 99.74 - 100%

It should be mentioned, that given the time series nature of the dataset, it is not a safe assumption to ignore outliers. By training respective models on outlier insensitive dataset, we would invite a potential problem, which risks blinding any models we train to future predicted spikes of activity.

In [ ]:
def get_outliers(df=None, low_quartile_limit=.01, upper_quartile_limit=.99, headers=None):
    """
    Detect and return which rows are considered outliers within the dataset, determined by :quartile_limit (99%)
    """
    outlier_rows = [] # This list of lists consists of elements of the following notation [column,rowid]
    for header in headers:
        try:
            df[header] = df[header].astype(float)
            q = df[header].quantile(upper_quartile_limit)
            series_row = (df[df[header] > q].index)
            for id in list(np.array(series_row)):
                outlier_rows.append([header,id])
            q = df[header].quantile(low_quartile_limit)
            series_row = (df[df[header] < q].index)
            for id in list(np.array(series_row)):
                outlier_rows.append([header,id])
        except Exception as e:
            print(str(e))
    
    unique_ids = []
    unique_outlier_rows = []
    
    for col, rowid in outlier_rows:
        if rowid not in unique_ids:
            unique_outlier_rows.append([col,rowid])
            unique_ids.append(rowid)
    return unique_outlier_rows

def remove_outliers(df=None, low_quartile_limit=.01, upper_quartile_limit=.99, headers=None):
    """
    Remove rows which are considered outliers within the dataset, determined by :quartile_limit (99%)
    """
    length_before = len(df)
    outliers_index = []
    for header in headers:
        try:
            df[header] = df[header].astype(float)
            q = df[header].quantile(upper_quartile_limit)
            outliers_index.append(list(np.array(df[df[header] > q].index)))
            q = df[header].quantile(low_quartile_limit)
            outliers_index.append(list(np.array(df[df[header] < q].index)))
        except Exception as e:
            print(str(e))
    #flat_outliers_index = [item for sublist in l for item in outliers_index]
    flat_outliers_index = [item for sublist in outliers_index for item in sublist]
    outliers_index = list(set(flat_outliers_index))
    df = df.drop(outliers_index)
    return df

# Defining which columns will be exposed to outliers
rep_hist_sysmetric_summary_header_outliers = rep_hist_sysmetric_summary_headers

# Printing dataframe before adjustments
print('\n\nDATAFRAMES WITH OUTLIERS')
print(rep_hist_sysmetric_summary_df.shape)
print('----------------------------')

#Printing outliers to screen
rep_hist_sysmetric_summary_df_outliers = get_outliers(df=rep_hist_sysmetric_summary_df, headers=rep_hist_sysmetric_summary_header_outliers,upper_quartile_limit=upper_quartile_limit,low_quartile_limit=low_quartile_limit)
print('\n\nOUTLIERS')
print(len(rep_hist_sysmetric_summary_df_outliers))
print('----------------------------')

# Dropping Outliers
rep_hist_sysmetric_summary_df_pruned = remove_outliers(df=rep_hist_sysmetric_summary_df, headers=rep_hist_sysmetric_summary_header_outliers,upper_quartile_limit=upper_quartile_limit,low_quartile_limit=low_quartile_limit)
print('\n\nDATAFRAMES WITHOUT OUTLIERS')
print(rep_hist_sysmetric_summary_df_pruned.shape)

### Plotting data distribution without outliers

Plotting metrics against SNAP_ID, without outliers.

In [ ]:
if debug_mode is False:
    
    # Plotting Histograms without Outliers
    for header in rep_hist_sysmetric_summary_header_outliers:
        print('REP_HIST_SYSMETRIC - ' + header + ' - STRIPPED HISTOGRAM')
        plot_hist(df=rep_hist_sysmetric_summary_df_pruned, tpc_type=tpcds, table='rep_hist_sysmetric_summary', feature_column=header, bin_size=10)
    
    # Plotting Scatter Plots without Outliers
    for header in rep_hist_sysmetric_summary_header_outliers:
        print('REP_HIST_SYSMETRIC - ' + header + ' - STRIPPED SCATTER')
        plot_scatter(df=rep_hist_sysmetric_summary_df_pruned, tpc_type=tpcds,table='rep_hist_sysmetric_summary',feature_column=header)

### Dropping Redundant Columns

Dropping redundant columns which are not detrimental to the task at hand (NB: This is only the first steps towards feature selection. This step ensures that specific columns which are SURELY not useful are dropped ahead of time).

http://benalexkeen.com/feature-scaling-with-scikit-learn/

Based on the above plots, one can argue that the data distribution is uneven, and does not correlate to any particular pattern. A normalization approach (MinMaxScaling and/or RobustScaling) to the presented dataset is a more likely candidate than standardizing of the presented dataset. 

Reasons behind normalizing the dataset rather than standardizing, is due to the vast standard deviations from the mean for several feature columns.

In [ ]:
retain_headers_rep_hist_sysmetric_summary = rep_hist_sysmetric_summary_headers

def retain_these_columns(dataframe, headers):
    """
    Drops all columns from dataframe except those passed in the header
    """
    dataframe.reset_index(inplace=True)
    return dataframe[headers]

shape = rep_hist_sysmetric_summary_df_pruned.shape
rep_hist_sysmetric_summary_df_pruned = retain_these_columns(dataframe=rep_hist_sysmetric_summary_df_pruned,headers=retain_headers_rep_hist_sysmetric_summary)
print('Before: ' + str(shape) + '| After: ' + str(rep_hist_sysmetric_summary_df_pruned.shape))

### Dataset Consolidation

Grouping datasets on SNAP_ID

In [ ]:
rep_hist_sysmetric_summary_df_pruned = rep_hist_sysmetric_summary_df_pruned[rep_hist_sysmetric_summary_df_pruned['SNAP_ID'] > 0]

# Group By Values by SNAP_ID , sum all metrics (for table REP_HIST_SNAPSHOT)
rep_hist_sysmetric_summary_df_pruned = rep_hist_sysmetric_summary_df_pruned.groupby(['SNAP_ID']).sum()
rep_hist_sysmetric_summary_df_pruned.reset_index(inplace=True)

print("REP_HIST_SYSMETRIC_SUMMARY shape after transformation: " + str(rep_hist_sysmetric_summary_df_pruned.shape))

### Normalization

Relavent Sources:

* http://jmlr.csail.mit.edu/papers/volume3/guyon03a/guyon03a.pdf
* https://machinelearningmastery.com/rescaling-data-for-machine-learning-in-python-with-scikit-learn/

https://machinelearningmastery.com/normalize-standardize-time-series-data-python/ recommends a normalization preprocessing technique for data distribution that can closely approximate minimum and maximum observable values per column:

<i>"Normalization requires that you know or are able to accurately estimate the minimum and maximum observable values. You may be able to estimate these values from your available data. If your time series is trending up or down, estimating these expected values may be difficult and normalization may not be the best method to use on your problem."</i>

Normalization formula is stated as follows: $$y=(x-min)/(max-min)$$

### Standardization

https://machinelearningmastery.com/normalize-standardize-time-series-data-python/ recommends a standardization preprocessing technique for data distributions that observe a Gaussian spread, with a mean of 0 and a standard deviation of 1 (approximately close to these values):

<i>"Standardization assumes that your observations fit a Gaussian distribution (bell curve) with a well behaved mean and standard deviation. You can still standardize your time series data if this expectation is not met, but you may not get reliable results."</i>

Standardization formula is stated as follows: $$y=(x-mean)/StandardDeviation$$
Mean defined as: $$mean=sum(x)/count(x)$$
Standard Deviation defined as: $$StandardDeviation=sqrt(sum((x-mean)^2)/count(x))$$

In [ ]:
def robust_scaler(dataframe, headers):
    """
    Normalize df using interquartile ranges as min-max, this way outliers do not play a heavy emphasis on the
    normalization of values.
    """
    return preprocessing.robust_scale(dataframe)

def minmax_scaler(dataframe, headers):
    """
    Normalize df using min-max ranges for normalization method
    """
    return preprocessing.minmax_scale(dataframe)

def normalize(dataframe, headers):
    """
    The normalizer scales each value by dividing each value by its magnitude in n-dimensional space for n number of features. 
    """
    return preprocessing.normalize(dataframe)

print('------------------BEFORE------------------')
print('------------------REP_HIST_SYSMETRIC_SUMMARY------------------')
print(rep_hist_sysmetric_summary_df_pruned.shape)
print('\n')
#print(rep_hist_snapshot_df_pruned.head())
#
# ROBUST SCALER
# rep_hist_sysmetric_summary_df_pruned_norm = robust_scaler(dataframe=rep_hist_sysmetric_summary_df_pruned,
#                                                           headers=retain_headers_rep_hist_sysmetric_summary)
#
# MINMAX SCALER
# rep_hist_sysmetric_summary_df_pruned_norm = minmax_scaler(dataframe=rep_hist_sysmetric_summary_df_pruned,
#                                                           headers=retain_headers_rep_hist_sysmetric_summary)
#
# NORMALIZER
rep_hist_sysmetric_summary_df_pruned_norm = normalize(dataframe=rep_hist_sysmetric_summary_df_pruned,
                                                      headers=retain_headers_rep_hist_sysmetric_summary)

print('\n\n------------------AFTER------------------')
print('------------------REP_HIST_SYSMETRIC_SUMMARY------------------')
print(rep_hist_sysmetric_summary_df_pruned_norm.shape)
print('\n\n')
print('\n\nREP_HIST_SYSMETRIC_SUMMARY')
print(rep_hist_sysmetric_summary_df_pruned_norm[0])

# Filter Methods

* https://machinelearningmastery.com/feature-selection-machine-learning-python/

Filter methods allow for the univariate analysis of features. Particularly, the following statistical methods have been considered and dismissed as explained below:

* Information Gain - Data at hand is continous by nature, whilst MI is usually applicable against discrete values / binned labels.
* Pearson Correlation Coefficient - Applicable to data with linear distributions, which is not the case for the majority of the presented features.

### Chi2

Therefore, a likely candidate for a first univariate, filter test is a chi2 measure, between X labels 'SNAP_ID', and other output 'y' labels. NB: Due to chi2 requiring non-nagative values, a data normalization strategy was opted for (Refer to above cell).

The following cell computes the chi2 value for each feature pertaining in the following tables, in relation to the SNAP_ID feature:
* REP_HIST_SYSMETRIC_SUMMARY

In [ ]:
def chi2_test(top_n_features=30, X_df=None, y_df=None, table=None, headers=None):
    """
    Carries out a chi squared test on passed X,y dataframes, selecting top N features ranked by highest scoring
    """
    chi2_selector = SelectKBest(score_func=chi2, k=top_n_features)
    X_kbest = chi2_selector.fit_transform(X_df, y_df)
    print('\n\nTable [' + table.upper() + ']')
    print('Original number of features: ' + str(X_df.shape[1]))
    print('Reduced number of features: ' + str(X_kbest.shape[1]) + "\n")
    outcome = chi2_selector.get_support()
    
    scoring_sheet = []
    for i in range(0,len(headers)-1):
        if outcome[i]:
            scoring_sheet.append([headers[i],chi2_selector.scores_[i]])
    
    scoring_sheet = sorted(scoring_sheet, key=itemgetter(1), reverse=True)
    [print('Feature [' + str(row) + '] with score [' + str(score) + ']') for row, score in scoring_sheet[:top_n_features]]
    
    scoring_sheet = pd.Series((v[1] for v in scoring_sheet[:top_n_features]) )
    scoring_sheet[:top_n_features].plot.bar()
    plt.ylabel('CHI2 RANKING')
    plt.xlabel('FEATURES')
    plt.title('Features Ranked By Chi2 Scoring')
    plt.rcParams['figure.figsize'] = [20, 15]
    plt.show()

chi2_test(top_n_features=top_n_features, X_df=rep_hist_sysmetric_summary_df_pruned_norm, y_df=rep_hist_sysmetric_summary_df_pruned['SNAP_ID'], table='REP_HIST_SYSMETRIC_SUMMARY', headers=retain_headers_rep_hist_sysmetric_summary)

### Random Forest Feature Importance

Calculating MI (Mutual Information) scoring between data matrix X (feature vectors) and target column y ('SNAP_ID') 

In [ ]:
def rfr_ranking(X_df=None, y_df=None, headers=None, top_n_features=30, parallel_degree=1):
    """
    Ranks features using a filter RFR method, and plots them in descending order (ranked by importance)
    """
    X_df = np.array(X_df)
    y_df = np.array(y_df)
    rfr = RandomForestRegressor(n_estimators=2000,
                                n_jobs=parallel_degree)
    rfr.fit(X_df, y_df)
    importances = pd.DataFrame({'feature':headers,
                                'importance':np.round(rfr.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    print(importances[:top_n_features])
    importances[:top_n_features].plot.bar()
    plt.ylabel('FEATURE INFORMATION IMPORTANCE')
    plt.xlabel('FEATURES')
    plt.title('Features Ranked By RFC Scoring')
    plt.rcParams['figure.figsize'] = [20, 15]
    plt.show()

rfr_ranking(top_n_features=top_n_features,
            X_df=rep_hist_sysmetric_summary_df_pruned_norm, 
            y_df=rep_hist_sysmetric_summary_df_pruned['SNAP_ID'], 
            headers=retain_headers_rep_hist_sysmetric_summary,
            parallel_degree=parallel_degree)

### Wrapper Methods

Use a number of machine learning models to evaluate features together and rank by highest. The following machine learning models will be opted for:

* Random Forest Classifier
* Gradient Boosting

In a 'Brute-Fore' approach, these machine learning heuristics will strip away 1 feature at a time in a method referred to as 'Recursive Feature Elimination', and compare accuracy with every variable elimination. This allows the respective classifier to establish an optimum feature configuration with the highest accuracy score.

https://www.fabienplisson.com/choosing-right-features/

### Random Forest Wrapper (Feature Combination)  (Regression)

In [ ]:
def rfr_wrapper(X_df=None, y_df=None, test_split=.4, random_state=42, table_name=None, top_n_features=10, parallel_degree=1):
    """
    Random Forest Regressor - Takes data matrix and target vector, and evaluates best combination of features 
    using an RFR model.
    """
    X_df = np.array(X_df)
    y_df = np.array(y_df)
    #feature_count = len(X_df[0])
    #print(len(X_df[0]))
    val_op, optimum_features = 0, 0
    val_op = 0
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=test_split)
    model = RandomForestRegressor(n_estimators=top_n_features, 
                                  n_jobs=parallel_degree)
    model.fit(X_train, y_train)
    
    # make predictions for test data and evaluate
    pred_y = model.predict(X_test)
    predictions = [round(value) for value in pred_y]
    r2s = r2_score(y_test, predictions)
    print("Table [" + table_name + "] RFR R2 Score: " + str(r2s))
    
    # fit model using each importance as a threshold
    print('Feature Importance\n' + str('-'*60))
    print(model.feature_importances_)
    print(str('-'*60))
    thresholds = np.sort(model.feature_importances_)
    feature_counts, feature_score = [],[]
    for thresh in thresholds:
        # selecting features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_train_x = selection.transform(X_train)
        
        # training model
        selection_model = RandomForestRegressor(n_estimators=top_n_features,
                                                n_jobs=parallel_degree)
        selection_model.fit(select_train_x, y_train)
        
        # evaluating model
        select_test_x = selection.transform(X_test)
        pred_y = selection_model.predict(select_test_x)
        predictions = [round(value) for value in pred_y]
        r2s = r2_score(y_test, predictions)
        print("Thresh=" + str(thresh) + ", n=" + str(select_train_x.shape[1]) + ", R2 Score: " + str(r2s))
        if(r2s > val_op):
            val_op = r2s
            optimum_features = select_train_x.shape[1]
            
        # Add/Keep track of '[no of features','r2 score']
        feature_counts.append(select_train_x.shape[1])
        feature_score.append(r2s)
        
    # Plot feature count performance
    plt.rcParams['figure.figsize'] = [20, 15]
    plt.xlabel('Features')
    plt.ylabel('R2 Score')
    plt.title('Feature Pairing Performance')
    plt.plot(feature_counts, feature_score)
    plt.show()
    
    return val_op, optimum_features

rfr_hist_sysmetric_summary_score, rfr_hist_sysmetric_summary_count = rfr_wrapper(X_df=rep_hist_sysmetric_summary_df_pruned_norm,
                                                                                 y_df=rep_hist_sysmetric_summary_df_pruned['SNAP_ID'],
                                                                                 test_split=test_split,
                                                                                 table_name='REP_HIST_SYSMETRIC_SUMMARY',
                                                                                 top_n_features=top_n_features,
                                                                                 parallel_degree=parallel_degree)

### Gradient Boosting Wrapper (Feature Combination)  (Regression)

https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/

In [ ]:
def gradient_boosting_wrapper(X_df=None, y_df=None, test_split=.4, random_state=42, table_name=None, top_n_features=10):
    """
    Gradient Boosting Regressor - Takes data matrix and target vector, and evaluates best combination of features 
    using a GBR model.
    """
    X_df = np.array(X_df)
    y_df = np.array(y_df)
    #feature_count = len(X_df[0])
    val_op, optimum_features = 0, 0
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=test_split)
    model = GradientBoostingRegressor(n_estimators=top_n_features)
    model.fit(X_train, y_train)
    
    # make predictions for test data and evaluate
    pred_y = model.predict(X_test)
    predictions = [round(value) for value in pred_y]
    r2s = r2_score(y_test, predictions)
    print("Table [" + table_name + "] RFR R2 Score: " + str(r2s))
    
    # fit model using each importance as a threshold
    print('Feature Importance\n' + str('-'*60))
    print(model.feature_importances_)
    print(str('-'*60))
    thresholds = np.sort(model.feature_importances_)
    feature_counts, feature_score = [],[]
    for thresh in thresholds:
        # selecting features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_train_x = selection.transform(X_train)
        
        # training model
        selection_model = GradientBoostingRegressor(n_estimators=top_n_features)
        selection_model.fit(select_train_x, y_train)
        
        # evaluating model
        select_test_x = selection.transform(X_test)
        pred_y = selection_model.predict(select_test_x)
        predictions = [round(value) for value in pred_y]
        r2s = r2_score(y_test, predictions)
        print("Thresh=" + str(thresh) + ", n=" + str(select_train_x.shape[1]) + ", R2 Score: " + str(r2s))
        if(r2s > val_op):
            val_op = r2s
            optimum_features = select_train_x.shape[1]
            
        # Add/Keep track of '[no of features','r2 score']
        feature_counts.append(select_train_x.shape[1])
        feature_score.append(r2s)
    
    # Plot feature count performance
    plt.rcParams['figure.figsize'] = [20, 15]
    plt.xlabel('Features')
    plt.ylabel('R2 Score')
    plt.title('Feature Pairing Performance')
    plt.plot(feature_counts, feature_score)
    plt.show()
    
    return val_op, optimum_features

gbw_hist_sysmetric_summary_score, gbw_hist_sysmetric_summary_count = gradient_boosting_wrapper(X_df=rep_hist_sysmetric_summary_df_pruned_norm,
                                                                                               y_df=rep_hist_sysmetric_summary_df_pruned['SNAP_ID'],
                                                                                               test_split=test_split,
                                                                                               table_name='REP_HIST_SYSMETRIC_SUMMARY',
                                                                                               top_n_features=top_n_features)

### Recursive Feature Elimination (Regression)

In [ ]:
if rfr_hist_sysmetric_summary_score > gbw_hist_sysmetric_summary_score:
    rep_hist_sysmetric_summary_op = rfr_hist_sysmetric_summary_count
    rep_hist_sysmetric_summary_model = 0
else:
    rep_hist_sysmetric_summary_op = gbw_hist_sysmetric_summary_count
    rep_hist_sysmetric_summary_model = 1

def rfe_selector(X_df=None, y_df=None, test_split=.4, random_state=42, table_name=None, optimum_feature_count=0, model=None, top_n_features=10, parallel_degree=1):
    """
    Recursive Feature Elimination Function
    """
    X_df = np.array(X_df)
    y_df = np.array(y_df)
    #feature_count = len(X_df[0])
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=test_split)
    if model == 0:
        model = RandomForestRegressor(n_estimators=top_n_features,
                                      n_jobs=parallel_degree)
    elif model == 1:
        model = GradientBoostingRegressor(n_estimators=top_n_features)
    
    # create the RFE model and select 4 attributes
    rfe_model = RFE(model, optimum_feature_count, step=1)
    rfe_model = rfe_model.fit(X_train, y_train)
    
    # summarize the selection of the attributes
    print(rfe_model.support_)
    print(rfe_model.ranking_)
    
    # evaluate the model on testing set
    pred_y = rfe_model.predict(X_test)
    predictions = [round(value) for value in pred_y]
    r2s = r2_score(y_test, predictions)
    print("Table [" + table_name + "] RFR R2 Score: " + str(r2s) + " with " + str(optimum_feature_count) + " features")
    print("\n\n------------------------------------------\n\n")
    
rfe_selector(X_df=rep_hist_sysmetric_summary_df_pruned_norm,
             y_df=rep_hist_sysmetric_summary_df_pruned['SNAP_ID'],
             test_split=test_split,
             table_name='REP_HIST_SYSMETRIC_SUMMARY',
             optimum_feature_count=rep_hist_sysmetric_summary_op,
             model = rep_hist_sysmetric_summary_model,
             top_n_features=top_n_features,
             parallel_degree=parallel_degree)